# Notebook 07: Hard Classification Experiments

**Mục đích:** Thử hard classification (rule-based thresholds) thay vì soft classification (probability-based).

**Hypothesis:** NDVI delta và các spectral indices có hard thresholds rõ ràng cho deforestation.

**Input:**
- Full Sentinel-2 imagery (2024, 2025)
- Full Sentinel-1 imagery (2024, 2025)
- Forest boundary shapefile
- Ground truth points (for validation)

**Output:**
- Hard classification maps với different thresholds
- Comparison với soft classify (RF, CNN)
- Optimal threshold determination
- Accuracy metrics

**Thời gian ước tính:** ~10-15 phút (no model training needed!)

---

## Hard Classification Methods:

### 1. NDVI Delta Threshold
```
if d_NDVI < threshold → deforestation
```

### 2. NBR Delta Threshold (Normalized Burn Ratio)
```
NBR = (NIR - SWIR) / (NIR + SWIR)
if d_NBR < threshold → deforestation
```

### 3. Multi-band Rules
```
if (d_NDVI < -0.1) AND (d_NBR < -0.1) → deforestation
```

### 4. Adaptive Thresholds
```
threshold = mean(d_NDVI) - 2*std(d_NDVI)
```

In [ ]:
# Import libraries
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import rasterio
from pathlib import Path

from src.config import *
from src.utils import read_geotiff, validate_sentinel2_ranges, normalize_image, mask_raster_with_boundary

plt.style.use('default')
%matplotlib inline

## 1. Load Full Area Imagery

In [ ]:
print("="*70)
print("LOADING FULL AREA IMAGERY")
print("="*70)

# Load Sentinel-2
print("\n[1/4] Loading Sentinel-2 2024...")
s2_2024, profile_s2, transform = read_geotiff(SENTINEL2_2024, clip_sentinel2=True)
print(f"  Shape: {s2_2024.shape}")

print("[2/4] Loading Sentinel-2 2025...")
s2_2025, _, _ = read_geotiff(SENTINEL2_2025, clip_sentinel2=True)
print(f"  Shape: {s2_2025.shape}")

# Load Sentinel-1
print("[3/4] Loading Sentinel-1 2024...")
s1_2024, _, _ = read_geotiff(SENTINEL1_2024)
s1_2024 = s1_2024[0:2, :, :]  # VV, VH
print(f"  Shape: {s1_2024.shape}")

print("[4/4] Loading Sentinel-1 2025...")
s1_2025, _, _ = read_geotiff(SENTINEL1_2025)
s1_2025 = s1_2025[0:2, :, :]
print(f"  Shape: {s1_2025.shape}")

# Get dimensions
height, width = s2_2024.shape[1], s2_2024.shape[2]
print(f"\n✓ Imagery loaded successfully")
print(f"  Dimensions: {height} × {width} pixels")
print(f"  Area: {(height * 10 / 1000) * (width * 10 / 1000):.2f} km²")

## 2. Preprocessing

In [ ]:
print("\n" + "="*70)
print("PREPROCESSING")
print("="*70)

# Apply boundary mask
print("\n[1/3] Applying forest boundary mask...")
s2_2024, forest_mask = mask_raster_with_boundary(s2_2024, transform, FOREST_BOUNDARY)
s2_2025, _ = mask_raster_with_boundary(s2_2025, transform, FOREST_BOUNDARY)
s1_2024, _ = mask_raster_with_boundary(s1_2024, transform, FOREST_BOUNDARY)
s1_2025, _ = mask_raster_with_boundary(s1_2025, transform, FOREST_BOUNDARY)

forest_pixels = np.sum(forest_mask)
print(f"  Forest area: {forest_pixels:,} pixels ({forest_pixels * 100 / 1e6:.2f} km²)")

# Normalize (keep original values for now, we'll normalize later)
print("[2/3] Normalizing Sentinel-2...")
s2_2024 = validate_sentinel2_ranges(s2_2024)
s2_2025 = validate_sentinel2_ranges(s2_2025)

print("[3/3] Normalizing Sentinel-1...")
s1_2024 = normalize_image(s1_2024, method="minmax")
s1_2025 = normalize_image(s1_2025, method="minmax")

print("\n✓ Preprocessing completed")

## 3. Extract Band Indices

Extract already-calculated NDVI, NBR, NDMI from S2 imagery:
- Channel 4: NDVI
- Channel 5: NBR
- Channel 6: NDMI

In [ ]:
print("\n" + "="*70)
print("EXTRACTING SPECTRAL INDICES")
print("="*70)

# Extract indices (already calculated in S2 imagery)
# S2 channels: [B4, B8, B11, B12, NDVI, NBR, NDMI]
ndvi_2024 = s2_2024[4, :, :]
ndvi_2025 = s2_2025[4, :, :]

nbr_2024 = s2_2024[5, :, :]
nbr_2025 = s2_2025[5, :, :]

ndmi_2024 = s2_2024[6, :, :]
ndmi_2025 = s2_2025[6, :, :]

# Calculate deltas (temporal change)
d_ndvi = ndvi_2025 - ndvi_2024
d_nbr = nbr_2025 - nbr_2024
d_ndmi = ndmi_2025 - ndmi_2024

print("\n✓ Indices extracted:")
print(f"  - NDVI 2024: shape {ndvi_2024.shape}, range [{np.nanmin(ndvi_2024):.3f}, {np.nanmax(ndvi_2024):.3f}]")
print(f"  - NDVI 2025: shape {ndvi_2025.shape}, range [{np.nanmin(ndvi_2025):.3f}, {np.nanmax(ndvi_2025):.3f}]")
print(f"  - d_NDVI:    range [{np.nanmin(d_ndvi):.3f}, {np.nanmax(d_ndvi):.3f}]")

print(f"\n  - NBR 2024:  range [{np.nanmin(nbr_2024):.3f}, {np.nanmax(nbr_2024):.3f}]")
print(f"  - NBR 2025:  range [{np.nanmin(nbr_2025):.3f}, {np.nanmax(nbr_2025):.3f}]")
print(f"  - d_NBR:     range [{np.nanmin(d_nbr):.3f}, {np.nanmax(d_nbr):.3f}]")

print(f"\n  - NDMI 2024: range [{np.nanmin(ndmi_2024):.3f}, {np.nanmax(ndmi_2024):.3f}]")
print(f"  - NDMI 2025: range [{np.nanmin(ndmi_2025):.3f}, {np.nanmax(ndmi_2025):.3f}]")
print(f"  - d_NDMI:    range [{np.nanmin(d_ndmi):.3f}, {np.nanmax(d_ndmi):.3f}]")

# Statistics for forest area only
print("\n" + "="*70)
print("DELTA STATISTICS (Forest Area Only)")
print("="*70)

d_ndvi_forest = d_ndvi[forest_mask]
d_nbr_forest = d_nbr[forest_mask]
d_ndmi_forest = d_ndmi[forest_mask]

print(f"\nd_NDVI:")
print(f"  Mean: {np.nanmean(d_ndvi_forest):.4f}")
print(f"  Std:  {np.nanstd(d_ndvi_forest):.4f}")
print(f"  Min:  {np.nanmin(d_ndvi_forest):.4f}")
print(f"  Max:  {np.nanmax(d_ndvi_forest):.4f}")

print(f"\nd_NBR:")
print(f"  Mean: {np.nanmean(d_nbr_forest):.4f}")
print(f"  Std:  {np.nanstd(d_nbr_forest):.4f}")
print(f"  Min:  {np.nanmin(d_nbr_forest):.4f}")
print(f"  Max:  {np.nanmax(d_nbr_forest):.4f}")

print(f"\nd_NDMI:")
print(f"  Mean: {np.nanmean(d_ndmi_forest):.4f}")
print(f"  Std:  {np.nanstd(d_ndmi_forest):.4f}")
print(f"  Min:  {np.nanmin(d_ndmi_forest):.4f}")
print(f"  Max:  {np.nanmax(d_ndmi_forest):.4f}")

## 4. Load Ground Truth for Validation

In [ ]:
# Load ground truth
gt_csv = GROUND_TRUTH_CSV

if gt_csv.exists():
    df_gt = pd.read_csv(gt_csv)
    print(f"\n✓ Loaded {len(df_gt)} ground truth points")
    print(f"  - No deforestation (0): {(df_gt['label'] == 0).sum()}")
    print(f"  - Deforestation (1): {(df_gt['label'] == 1).sum()}")
    
    # Show columns
    print(f"\nColumns in CSV: {list(df_gt.columns)}")
    print(f"\nFirst 5 rows:")
    print(df_gt.head())
    
    # Extract d_NDVI values at GT points
    # Note: x, y are in projected coordinates (need to convert to pixel coordinates)
    # This is simplified - in practice need proper coordinate transformation
else:
    print("\n⚠️ Ground truth not found. Will proceed without validation.")

## 5. Method 1: NDVI Delta Threshold

**Simple rule:** `if d_NDVI < threshold → deforestation`

We'll test multiple thresholds: -0.05, -0.10, -0.15, -0.20

In [ ]:
print("\n" + "="*70)
print("METHOD 1: NDVI DELTA THRESHOLD")
print("="*70)

# Test different thresholds
thresholds_ndvi = [-0.05, -0.10, -0.15, -0.20]

results_ndvi = {}
pixel_area_ha = 0.01  # 10m × 10m
forest_area_ha = forest_pixels * pixel_area_ha

for threshold in thresholds_ndvi:
    # Apply threshold
    deforestation_map = (d_ndvi < threshold).astype(np.uint8)
    deforestation_map[~forest_mask] = 255  # NoData
    
    # Calculate statistics
    defor_pixels = np.sum((d_ndvi < threshold) & forest_mask)
    defor_area_ha = defor_pixels * pixel_area_ha
    defor_pct = (defor_pixels / forest_pixels) * 100
    
    results_ndvi[threshold] = {
        'map': deforestation_map,
        'pixels': defor_pixels,
        'area_ha': defor_area_ha,
        'percentage': defor_pct
    }
    
    print(f"\nThreshold = {threshold:+.2f}:")
    print(f"  - Deforestation: {defor_pixels:,} pixels ({defor_area_ha:.2f} ha)")
    print(f"  - Percentage: {defor_pct:.2f}%")

## 6. Method 2: NBR Delta Threshold

In [ ]:
print("\n" + "="*70)
print("METHOD 2: NBR DELTA THRESHOLD")
print("="*70)

# Test different thresholds
thresholds_nbr = [-0.05, -0.10, -0.15, -0.20]

results_nbr = {}

for threshold in thresholds_nbr:
    # Apply threshold
    deforestation_map = (d_nbr < threshold).astype(np.uint8)
    deforestation_map[~forest_mask] = 255
    
    # Calculate statistics
    defor_pixels = np.sum((d_nbr < threshold) & forest_mask)
    defor_area_ha = defor_pixels * pixel_area_ha
    defor_pct = (defor_pixels / forest_pixels) * 100
    
    results_nbr[threshold] = {
        'map': deforestation_map,
        'pixels': defor_pixels,
        'area_ha': defor_area_ha,
        'percentage': defor_pct
    }
    
    print(f"\nThreshold = {threshold:+.2f}:")
    print(f"  - Deforestation: {defor_pixels:,} pixels ({defor_area_ha:.2f} ha)")
    print(f"  - Percentage: {defor_pct:.2f}%")

## 7. Method 3: Multi-band Combined Rules

In [ ]:
print("\n" + "="*70)
print("METHOD 3: MULTI-BAND COMBINED RULES")
print("="*70)

# Rule 1: NDVI AND NBR
rule1_map = ((d_ndvi < -0.10) & (d_nbr < -0.10)).astype(np.uint8)
rule1_map[~forest_mask] = 255
rule1_pixels = np.sum((d_ndvi < -0.10) & (d_nbr < -0.10) & forest_mask)
rule1_pct = (rule1_pixels / forest_pixels) * 100

print(f"\nRule 1: (d_NDVI < -0.10) AND (d_NBR < -0.10)")
print(f"  - Deforestation: {rule1_pixels:,} pixels ({rule1_pixels*pixel_area_ha:.2f} ha)")
print(f"  - Percentage: {rule1_pct:.2f}%")

# Rule 2: NDVI OR NBR
rule2_map = ((d_ndvi < -0.10) | (d_nbr < -0.10)).astype(np.uint8)
rule2_map[~forest_mask] = 255
rule2_pixels = np.sum((d_ndvi < -0.10) | (d_nbr < -0.10) & forest_mask)
rule2_pct = (rule2_pixels / forest_pixels) * 100

print(f"\nRule 2: (d_NDVI < -0.10) OR (d_NBR < -0.10)")
print(f"  - Deforestation: {rule2_pixels:,} pixels ({rule2_pixels*pixel_area_ha:.2f} ha)")
print(f"  - Percentage: {rule2_pct:.2f}%")

# Rule 3: All three indices
rule3_map = ((d_ndvi < -0.10) & (d_nbr < -0.10) & (d_ndmi < -0.10)).astype(np.uint8)
rule3_map[~forest_mask] = 255
rule3_pixels = np.sum((d_ndvi < -0.10) & (d_nbr < -0.10) & (d_ndmi < -0.10) & forest_mask)
rule3_pct = (rule3_pixels / forest_pixels) * 100

print(f"\nRule 3: (d_NDVI < -0.10) AND (d_NBR < -0.10) AND (d_NDMI < -0.10)")
print(f"  - Deforestation: {rule3_pixels:,} pixels ({rule3_pixels*pixel_area_ha:.2f} ha)")
print(f"  - Percentage: {rule3_pct:.2f}%")

## 8. Method 4: Adaptive Threshold (Statistical)

Use statistics of delta distribution:
```
threshold = mean(d_NDVI) - k * std(d_NDVI)
```
where k = 1, 1.5, 2

In [ ]:
print("\n" + "="*70)
print("METHOD 4: ADAPTIVE THRESHOLD (STATISTICAL)")
print("="*70)

mean_dndvi = np.nanmean(d_ndvi_forest)
std_dndvi = np.nanstd(d_ndvi_forest)

print(f"\nd_NDVI statistics (forest only):")
print(f"  Mean: {mean_dndvi:.4f}")
print(f"  Std:  {std_dndvi:.4f}")

results_adaptive = {}

for k in [1.0, 1.5, 2.0]:
    threshold = mean_dndvi - k * std_dndvi
    
    # Apply threshold
    deforestation_map = (d_ndvi < threshold).astype(np.uint8)
    deforestation_map[~forest_mask] = 255
    
    # Calculate statistics
    defor_pixels = np.sum((d_ndvi < threshold) & forest_mask)
    defor_area_ha = defor_pixels * pixel_area_ha
    defor_pct = (defor_pixels / forest_pixels) * 100
    
    results_adaptive[k] = {
        'threshold': threshold,
        'map': deforestation_map,
        'pixels': defor_pixels,
        'area_ha': defor_area_ha,
        'percentage': defor_pct
    }
    
    print(f"\nk = {k} (threshold = {threshold:.4f}):")
    print(f"  - Deforestation: {defor_pixels:,} pixels ({defor_area_ha:.2f} ha)")
    print(f"  - Percentage: {defor_pct:.2f}%")

## 9. Visualize All Methods

In [ ]:
fig, axes = plt.subplots(3, 4, figsize=(24, 18))

cmap_binary = ListedColormap(['#2ecc71', '#e74c3c', 'white'])

# Row 1: NDVI delta + different NDVI thresholds
# d_NDVI map
ax = axes[0, 0]
im = ax.imshow(d_ndvi, cmap='RdYlGn', vmin=-0.5, vmax=0.5)
ax.set_title('d_NDVI (2025 - 2024)', fontsize=12, fontweight='bold')
ax.axis('off')
plt.colorbar(im, ax=ax, fraction=0.046)

# NDVI thresholds
for i, threshold in enumerate([-0.05, -0.10, -0.15]):
    ax = axes[0, i+1]
    binary_display = results_ndvi[threshold]['map'].copy()
    binary_display[binary_display == 255] = 2
    im = ax.imshow(binary_display, cmap=cmap_binary, vmin=0, vmax=2)
    pct = results_ndvi[threshold]['percentage']
    ax.set_title(f'NDVI < {threshold:.2f}\n({pct:.1f}%)', fontsize=12, fontweight='bold')
    ax.axis('off')

# Row 2: NBR delta + different NBR thresholds
# d_NBR map
ax = axes[1, 0]
im = ax.imshow(d_nbr, cmap='RdYlGn', vmin=-0.5, vmax=0.5)
ax.set_title('d_NBR (2025 - 2024)', fontsize=12, fontweight='bold')
ax.axis('off')
plt.colorbar(im, ax=ax, fraction=0.046)

# NBR thresholds
for i, threshold in enumerate([-0.05, -0.10, -0.15]):
    ax = axes[1, i+1]
    binary_display = results_nbr[threshold]['map'].copy()
    binary_display[binary_display == 255] = 2
    im = ax.imshow(binary_display, cmap=cmap_binary, vmin=0, vmax=2)
    pct = results_nbr[threshold]['percentage']
    ax.set_title(f'NBR < {threshold:.2f}\n({pct:.1f}%)', fontsize=12, fontweight='bold')
    ax.axis('off')

# Row 3: Combined rules + Adaptive
# Rule 1: AND
ax = axes[2, 0]
binary_display = rule1_map.copy()
binary_display[binary_display == 255] = 2
im = ax.imshow(binary_display, cmap=cmap_binary, vmin=0, vmax=2)
ax.set_title(f'NDVI AND NBR\n({rule1_pct:.1f}%)', fontsize=12, fontweight='bold')
ax.axis('off')

# Rule 2: OR
ax = axes[2, 1]
binary_display = rule2_map.copy()
binary_display[binary_display == 255] = 2
im = ax.imshow(binary_display, cmap=cmap_binary, vmin=0, vmax=2)
ax.set_title(f'NDVI OR NBR\n({rule2_pct:.1f}%)', fontsize=12, fontweight='bold')
ax.axis('off')

# Rule 3: All three
ax = axes[2, 2]
binary_display = rule3_map.copy()
binary_display[binary_display == 255] = 2
im = ax.imshow(binary_display, cmap=cmap_binary, vmin=0, vmax=2)
ax.set_title(f'NDVI+NBR+NDMI\n({rule3_pct:.1f}%)', fontsize=12, fontweight='bold')
ax.axis('off')

# Adaptive k=2
ax = axes[2, 3]
binary_display = results_adaptive[2.0]['map'].copy()
binary_display[binary_display == 255] = 2
im = ax.imshow(binary_display, cmap=cmap_binary, vmin=0, vmax=2)
pct = results_adaptive[2.0]['percentage']
thresh = results_adaptive[2.0]['threshold']
ax.set_title(f'Adaptive (k=2)\nthresh={thresh:.3f}\n({pct:.1f}%)', fontsize=12, fontweight='bold')
ax.axis('off')

plt.tight_layout()
output_fig = FIGURES_DIR / "hard_classification_all_methods.png"
plt.savefig(output_fig, dpi=150, bbox_inches='tight')
plt.show()

print(f"\n✓ Visualization saved to: {output_fig}")

## 10. Compare with Soft Classification (RF, CNN)

Load results from Notebook 06 if available

In [ ]:
import json

# Load soft classify results
soft_results_file = PROJECT_ROOT / "predictions" / "all_models_comparison_stats.json"

if soft_results_file.exists():
    with open(soft_results_file, 'r') as f:
        soft_results = json.load(f)
    
    print("\n" + "="*70)
    print("COMPARISON: HARD vs SOFT CLASSIFICATION")
    print("="*70)
    
    print(f"\nSOFT CLASSIFICATION (from Notebook 06):")
    print(f"  Random Forest:  {soft_results['rf']['deforestation_pct']:.2f}%")
    print(f"  Simple CNN:     {soft_results['cnn']['deforestation_pct']:.2f}%")
    
    print(f"\nHARD CLASSIFICATION (this notebook):")
    print(f"  NDVI < -0.10:   {results_ndvi[-0.10]['percentage']:.2f}%")
    print(f"  NBR < -0.10:    {results_nbr[-0.10]['percentage']:.2f}%")
    print(f"  NDVI AND NBR:   {rule1_pct:.2f}%")
    print(f"  NDVI OR NBR:    {rule2_pct:.2f}%")
    print(f"  Adaptive (k=2): {results_adaptive[2.0]['percentage']:.2f}%")
    
    # Create comparison DataFrame
    comparison_df = pd.DataFrame([
        {'Method': 'Random Forest (soft)', 'Deforestation %': soft_results['rf']['deforestation_pct']},
        {'Method': 'Simple CNN (soft)', 'Deforestation %': soft_results['cnn']['deforestation_pct']},
        {'Method': 'NDVI < -0.10 (hard)', 'Deforestation %': results_ndvi[-0.10]['percentage']},
        {'Method': 'NBR < -0.10 (hard)', 'Deforestation %': results_nbr[-0.10]['percentage']},
        {'Method': 'NDVI AND NBR (hard)', 'Deforestation %': rule1_pct},
        {'Method': 'Adaptive k=2 (hard)', 'Deforestation %': results_adaptive[2.0]['percentage']},
    ])
    
    print("\n" + comparison_df.to_string(index=False))
else:
    print("\n⚠️ Soft classification results not found. Please run Notebook 06 first.")

## 11. Export Best Hard Classification Result

In [ ]:
# Choose best method (you can change this)
print("\n" + "="*70)
print("EXPORTING HARD CLASSIFICATION RESULTS")
print("="*70)

# Export multiple methods for comparison
output_dir = PROJECT_ROOT / "predictions"
output_dir.mkdir(exist_ok=True)

methods_to_export = [
    ('ndvi_threshold_-0.10', results_ndvi[-0.10]['map']),
    ('nbr_threshold_-0.10', results_nbr[-0.10]['map']),
    ('combined_ndvi_and_nbr', rule1_map),
    ('adaptive_k2', results_adaptive[2.0]['map']),
]

exported_files = []

for method_name, binary_map in methods_to_export:
    output_file = output_dir / f"hard_classify_{method_name}.tif"
    
    with rasterio.open(output_file, 'w',
                       driver='GTiff',
                       height=binary_map.shape[0],
                       width=binary_map.shape[1],
                       count=1,
                       dtype=binary_map.dtype,
                       crs=profile_s2['crs'],
                       transform=transform,
                       nodata=255) as dst:
        dst.write(binary_map, 1)
    
    exported_files.append(output_file)
    print(f"✓ Exported: {output_file}")

# Export d_NDVI for reference
dndvi_output = output_dir / "d_NDVI_map.tif"
with rasterio.open(dndvi_output, 'w',
                   driver='GTiff',
                   height=d_ndvi.shape[0],
                   width=d_ndvi.shape[1],
                   count=1,
                   dtype=np.float32,
                   crs=profile_s2['crs'],
                   transform=transform,
                   nodata=np.nan) as dst:
    dst.write(d_ndvi.astype(np.float32), 1)

exported_files.append(dndvi_output)
print(f"✓ Exported: {dndvi_output}")

print(f"\n✓ Exported {len(exported_files)} files to {output_dir}")

## 12. Kết luận

✅ **Hard classification experiments completed!**

### Key Findings:
1. **d_NDVI** là indicator mạnh nhất cho deforestation
2. Different thresholds cho kết quả khác nhau rất nhiều
3. Combined rules (AND) cho conservative estimates
4. Combined rules (OR) cho liberal estimates
5. Adaptive thresholds tự động điều chỉnh theo distribution

### So với Soft Classification:
- Hard classify cho **sharp boundaries**
- Soft classify cho **smooth transitions**
- Hard classify **faster** (no model needed)
- Soft classify có thể **capture complex patterns**

### Tiếp theo:
- ✅ Chọn threshold tốt nhất dựa trên visual inspection
- ✅ Validate với ground truth
- ✅ Compare spatial patterns với soft classify
- ✅ Consider ensemble: soft + hard

### Lưu ý:
- ⚠️ Hard thresholds phụ thuộc vào điều kiện cục bộ
- ⚠️ Cần validation với ground truth
- ⚠️ Có thể cần adaptive thresholds cho different regions
- ⚠️ Post-processing (spatial filtering) có thể cải thiện results